In [1]:
import sys, os, time
#%load_ext autoreload
#%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import Variable, grad
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
#%matplotlib notebook
%matplotlib tk

import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
from scipy.sparse import coo_matrix
#from envs import Discrete_Pos_Trap, Gym_Env, Print_Env
from torch.distributions.utils import lazy_property
import foundation as fd
import torch.multiprocessing as mp
import time

/home/fleeb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def periodic_padding(img, pad=1):
    N, C, H, W = img.size()
    
    pimg = torch.zeros(N, C, H+pad*2, W+pad*2).type_as(img)
    
    pimg[:,:,pad:-pad, pad:-pad] = img
    pimg[:,:,:pad,pad:-pad] = img[:,:,-pad:]
    pimg[:,:,-pad:,pad:-pad] = img[:,:,:pad]
    pimg[:,:,pad:-pad,:pad] = img[:,:,:,-pad:]
    pimg[:,:,pad:-pad,-pad:] = img[:,:,:,:pad]
    
    pimg[:,:,:pad,:pad] = img[:,:,-pad:,-pad:]
    pimg[:,:,:pad,-pad:] = img[:,:,-pad:,:pad]
    pimg[:,:,-pad:,:pad] = img[:,:,:pad,-pad:]
    pimg[:,:,-pad:,-pad:] = img[:,:,:pad,:pad]
    
    return pimg

In [3]:
plt.ion()

In [8]:
W, H = 128, 128
N = 1
K = 2

In [9]:
# glider = torch.zeros(10,10)
# glider[4,4] = 1
# glider[3,5] = 1
# glider[5,4] = 1
# glider[4,3] = 1
# glider[3,3] = 1

img = torch.rand(N,1,H,W)

In [10]:
order = torch.zeros(N,1,H,W)
order[:,:,:H//2] = 1

In [11]:
# norm
img = img / img.view(N, -1).sum(-1)
order = order / order.view(N, -1).sum(-1)

In [22]:
conv = nn.Conv2d(1,1, kernel_size=K*2+1, bias=False).cuda()

In [15]:
plt.figure()
plt.imshow(conv.weight[0,0].detach().cpu().numpy())

In [14]:
optim = torch.optim.RMSprop(conv.parameters(), lr=0.1, )#weight_decay=0.01)
criterion = nn.BCELoss()

In [16]:
def step(x):
    conv.weight.data = F.softmax(conv.weight.view(-1), 0).view(1,1,K*2+1,K*2+1)
    y = conv(periodic_padding(x, K))
    return y

In [21]:
x = img.clone().cuda()
y = order.clone().cuda()
plt.figure()
plt.imshow(x[0,0].detach().cpu().numpy())

In [23]:
plt.figure()

with torch.no_grad():
    for i in range(1000):
        plt.pause(0.02)
        plt.cla()
        plt.axis('off')
        plt.title(i+1)
        x = step(x)
        #loss = criterion(x, order)
        #optim.zero_grad()
        #loss.backward()
        #optim.step()

        plt.imshow(x[0,0].detach().cpu().numpy())

TclError: can't invoke "update" command: application has been destroyed

In [25]:
conv = torch.ones(3,3)
conv[1,1] = 0

In [33]:
x = glider.unsqueeze(0).unsqueeze(0)
c = conv.unsqueeze(0).unsqueeze(0)

In [68]:
y = F.conv2d(x, c, padding=1)
print(y.size())

torch.Size([1, 1, 10, 10])


In [46]:
def act(x,y): # non differentiable
    return ((y==3) * (x==0) + (x==1) * ((y==2) + (y==3))).float()

In [47]:
def step(x):
    return act(x, F.conv2d(x,c,padding=1))